In [1]:
import snappy
import csv
import pickle
import time

In [2]:
def change_notation(dt_code):
    """
    Changes Dowker-Thistlewait notation from alphabetical to numerical
    Input:
        dt_code (string): alphabetical DT notation
    Return:
        (string): numerical DT notation
    """
    alpha = "abcdefghijklmnopqrstuvwxyz"
    Alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    result = []
    for letter in dt_code:
        if letter in alpha:
            result.append(2* (alpha.index(letter) + 1))
        elif letter in Alpha:
            result.append(-2 * (Alpha.index(letter) + 1))
        else:
            print(dt_code)
    return "DT: " + str([tuple(result)])

In [3]:
start_time = time.time()
ALEX_GROUP=[]
alexander_polynomials=[]

for c in range(10):
    for K in snappy.HTLinkExteriors(knots_vs_links='knots',crossings=c):
        f=K.alexander_polynomial()
        if f not in alexander_polynomials:
            alexander_polynomials.append(f)
            ALEX_GROUP.append([f,[K.name()]])
        else:
            for [pol,knots] in ALEX_GROUP:
                if pol==f:
                    knots.append(K.name())

print('Number of different Alexander polynomials:',len(alexander_polynomials))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))    

Number of different Alexander polynomials: 79
Time taken: 2.0186439156532288 minutes 


In [5]:
start_time = time.time()
count=0
with open('16n-satellite.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        try:
            K=snappy.Link(change_notation(row[2])).exterior()
            f=K.alexander_polynomial()
            if f in alexander_polynomials:
                for [pol,knots] in ALEX_GROUP:
                    if pol==f:
                        knots.append(row[0])
                        count=count+1
        except IndexError:
            pass

print('Number of knots with the same Alexander polynomial as a knot with less than 10 crossings:',count)
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))   

dt_code
Number of knots with the same Alexander polynomial as a knot with less than 10 crossings: 4
Time taken: 0.0013364791870117188 minutes 


In [6]:
start_time = time.time()
count=0
with open('16n-torus.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        try:
            K=snappy.Link(change_notation(row[2])).exterior()
            f=K.alexander_polynomial()
            if f in alexander_polynomials:
                for [pol,knots] in ALEX_GROUP:
                    if pol==f:
                        knots.append(row[0])
                        count=count+1
        except IndexError:
            pass

print('Number of knots with the same Alexander polynomial as a knot with less than 10 crossings:',count)
print('Time taken: %s minutes ' % ((time.time() - start_time)/60)) 

dt_code
Number of knots with the same Alexander polynomial as a knot with less than 10 crossings: 0
Time taken: 7.850329081217448e-05 minutes 


In [8]:
start_time = time.time()
count=0
with open('16a-hyp.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        try:
            K=snappy.Link(change_notation(row[2])).exterior()
            f=K.alexander_polynomial()
            if f in alexander_polynomials:
                for [pol,knots] in ALEX_GROUP:
                    if pol==f:
                        knots.append(row[0])
                        count=count+1
        except IndexError:
            pass

print('Number of knots with the same Alexander polynomial as a knot with less than 10 crossings:',count)
print('Time taken: %s minutes ' % ((time.time() - start_time)/60)) 

dt_code
Number of knots with the same Alexander polynomial as a knot with less than 10 crossings: 0
Time taken: 308.9879988749822 minutes 


In [9]:
start_time = time.time()
count=0
with open('16n-hyp.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        try:
            K=snappy.Link(change_notation(row[2])).exterior()
            f=K.alexander_polynomial()
            if f in alexander_polynomials:
                for [pol,knots] in ALEX_GROUP:
                    if pol==f:
                        knots.append(row[0])
                        count=count+1
        except IndexError:
            pass

print('Number of knots with the same Alexander polynomial as a knot with less than 10 crossings:',count)
print('Time taken: %s minutes ' % ((time.time() - start_time)/60)) 

dt_code
Number of knots with the same Alexander polynomial as a knot with less than 10 crossings: 13195
Time taken: 274.82621866464615 minutes 


This is STRANGE. Why is non-alternating so much faster? Why don't we have the same Alexander polynomials for alternating?

In [10]:
with open('16Alexander_polynomials.pickle', 'wb') as f:
    pickle.dump(ALEX_GROUP, f)

We first work with the hyperbolic knots and just search if we find an isometry between their 0-surgeries.

In [14]:
nhyp16=[]
with open('16n-hyp.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        nhyp16.append(row)

In [15]:
def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_is_isometric_to(X,Y,return_isometries=False,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
     """ 
    if return_isometries==False:
        w='unclear'
        if X.homology()!=Y.homology():
            return False
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w==True:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=1,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=1,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                    if w==True:
                        return w
        return 'unclear'
    if return_isometries==True:
        w=False
        if X.homology()!=Y.homology():
            return []
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y,return_isometries)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w!=False:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,return_isometries,index=100,try_hard=False)
                    if w!=False:
                        return w
        return []

In [19]:
NOT_HYP=[['K3a1', 'SFS [S2: (2,1) (3,1) (6,-5)]'],
 ['K4a1', 'T x I / [ 2,1 | 1,1 ]'],
 ['K5a1', 'SFS [A: (2,1)] / [ 0,1 | 1,-1 ]'],
 ['K5a2', 'SFS [S2: (2,1) (5,2) (10,-9)]'],
 ['K6a3', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]'],
 ['K7a4', 'SFS [A: (3,2)] / [ 0,1 | 1,-1 ]'],
 ['K7a6',
  "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')])"],
 ['K7a7', 'SFS [S2: (2,1) (7,3) (14,-13)]'],
 ['K8n1', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ 0,1 | 1,0 ]'],
 ['K8n3', 'SFS [S2: (3,2) (4,1) (12,-11)]'],
 ['K8a11', 'SFS [A: (3,1)] / [ 0,1 | 1,-2 ]'],
 ['K8a18',
  "JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (2,1)]')]"],
 ['K9n5', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]'],
 ['K9a27', 'SFS [A: (4,3)] / [ 0,1 | 1,-1 ]'],
 ['K9a36',
  "JSJ[('SFSpace', 'SFS [A: (2,3)]'), ('SFSpace', 'SFS [A: (3,1)]')]"],
 ['K9a40', "JSJ[('hyperbolic', 'm202')]"],
 ['K9a41', 'SFS [S2: (2,1) (9,4) (18,-17)]'],
 ['K10n13', 'SFS [S2: (2,1) (5,1) (10,-7)]'],
 ['K10n21', 'SFS [S2: (3,1) (5,3) (15,-14)]'],
 ['K10n29',
  'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -1,2 | 0,1 ]'],
 ['K10a75', 'SFS [A: (4,1)] / [ 0,1 | 1,-2 ]'],
 ['K10a117',
  "JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,5)]')]"],
 ['K11n139', 'SFS [A: (2,1)] / [ 2,5 | 1,2 ]'],
 ['K11n141', "JSJ[('hyperbolic', 'm125')]"],
 ['K11a247', 'SFS [A: (5,4)] / [ 0,1 | 1,-1 ]'],
 ['K11a343',
  "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (4,7)]')])"],
 ['K11a362', "JSJ[('hyperbolic', 'm357')]"],
 ['K11a363',
  "JSJ[('SFSpace', 'SFS [A: (3,1)]'), ('SFSpace', 'SFS [A: (3,2)]')]"],
 ['K11a367', 'SFS [S2: (2,1) (11,5) (22,-21)]'],
 ['K12n121',
  'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]'],
 ['K12n582',
  'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -2,3 | -1,2 ]'],
 ['K12n721',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm043')]"],
 ['K12a803', 'SFS [A: (5,1)] / [ 0,1 | 1,-2 ]'],
 ['K12a1166',
  "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (4,1)]')])"],
 ['K12a1287',
  "JSJ[('SFSpace', 'SFS [A: (3,1)]'), ('SFSpace', 'SFS [A: (3,4)]')]"],
 ['K13n469',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm004')]"],
 ['K13a3143', 'SFS [A: (6,5)] / [ 0,1 | 1,-1 ]'],
 ['K13n3521', "JSJ[('hyperbolic', 'm329')]"],
 ['K13n3523', 'SFS [A: (2,1)] / [ 2,7 | 1,3 ]'],
 ['K13n3594', "JSJ[('hyperbolic', 'm292')]"],
 ['K13n3596',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,3)]')]"],
 ['K13n3663',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,1)]')]"],
 ['K13a4573',
  "JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (5,4)]')]"],
 ['K13n4587',
  'SFS [D: (2,1) (3,1)] U/m SFS [D: (2,1) (14,1)], m = [ -3,5 | -1,2 ]'],
 ['K13n4639',
  'SFS [D: (2,1) (3,1)] U/m SFS [D: (2,1) (10,1)], m = [ -5,7 | -2,3 ]'],
 ['K13a4843', "JSJ[('hyperbolic', 's548')]"],
 ['K13a4856',
  "JSJ[('SFSpace', 'SFS [A: (3,1)]'), ('SFSpace', 'SFS [A: (4,3)]')]"],
 ['K13a4873', "JSJ[('hyperbolic', 's876')]"],
 ['K13a4878', 'SFS [S2: (2,1) (13,6) (26,-25)]'],
 ['K14n3611', "JSJ[('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')]"],
 ['K14a12741', 'SFS [A: (6,1)] / [ 0,1 | 1,-2 ]'],
 ['K14a17730',
  "JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (5,9)]')]"],
 ['K14n18212',
  'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -3,4 | -2,3 ]'],
 ['K14n19265', "JSJ[('hyperbolic', 'v3319')]"],
 ['K14a19429',
  "JSJ[('SFSpace', 'SFS [A: (3,4)]'), ('SFSpace', 'SFS [A: (4,1)]')]"],
 ['K14n21881', 'SFS [S2: (3,2) (7,2) (21,-20)]'],
 ['K14n21882',
  'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (6,1)], m = [ 1,1 | 0,1 ]'],
 ['K14n22073',
  'SFS [D: (2,1) (3,1)] U/m SFS [A: (2,1) (2,1)] U/n SFS [D: (2,1) (3,2)], m = [ 1,-1 | 0,-1 ], n = [ 0,1 | 1,1 ]'],
 ['K14n22180',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,1)]')]"],
 ['K14n22185',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm137')]"],
 ['K14n22589', "JSJ[('hyperbolic', 'm129')]"],
 ['K14n24553',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 's663')]"],
 ['K14n26039',
  "JSJ[('SFSpace', 'Or, g=0 + 3 punctures x S1'), ('SFSpace', 'SFS [D: (2,1) (3,-2)]')]"],
 ['K15n19499',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm032')]"],
 ['K15n40211',
  'SFS [D: (2,1) (3,1)] U/m SFS [D: (2,1) (18,1)], m = [ -1,3 | 0,1 ]'],
 ['K15n41185', 'SFS [S2: (4,3) (5,1) (20,-19)]'],
 ['K15n43522', 'SFS [A: (2,1)] / [ 3,11 | 2,7 ]'],
 ['K15n48968', "JSJ[('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'v2817')]"],
 ['K15n51748',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm137')]"],
 ['K15a54894', 'SFS [A: (7,6)] / [ 0,1 | 1,-1 ]'],
 ['K15n59184',
  "JSJ[('SFSpace', 'Or, g=0 + 3 punctures x S1'), ('SFSpace', 'SFS [D: (2,1) (3,-2)]')]"],
 ['K15n72303', "JSJ[('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 's493')]"],
 ['K15a78880',
  "JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (6,5)]')]"],
 ['K15a84844', "JSJ[('hyperbolic', 'v1203')]"],
 ['K15a84969',
  "JSJ[('SFSpace', 'SFS [A: (3,1)]'), ('SFSpace', 'SFS [A: (5,4)]')]"],
 ['K15a85213', "JSJ[('hyperbolic', 'v2601')]"],
 ['K15a85234',
  "JSJ[('SFSpace', 'SFS [A: (4,1)]'), ('SFSpace', 'SFS [A: (4,7)]')]"],
 ['K15a85257', "JSJ[('hyperbolic', 'v3461')]"],
 ['K15a85263', 'SFS [S2: (2,1) (15,7) (30,-29)]'],
 ['K15n112477', "JSJ[('hyperbolic', 's503')]"],
 ['K15n112479', 'SFS [A: (2,1)] / [ 2,9 | 1,4 ]'],
 ['K15n113773', "JSJ[('hyperbolic', 's843')]"],
 ['K15n113775', "JSJ[('hyperbolic', 'm129')]"],
 ['K15n113923', "JSJ[('hyperbolic', 's441')]"],
 ['K15n115375', "JSJ[('hyperbolic', 'm129')]"],
 ['K15n115646',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('SFSpace', 'SFS [Or, g=0 + 3 punctures: (1,1)]')]"],
 ['K15n124802',
  'SFS [D: (2,1) (3,1)] U/m SFS [D: (2,1) (6,1)], m = [ -7,9 | -3,4 ]'],
 ['K15n142188',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('hyperbolic', None)]"],
 ['K15n153789',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm032')]"],
 ['K15n156076',
  "JSJ[('SFSpace', 'SFS [D: (2,1) (3,-2)]'), ('hyperbolic', None)]"],
 ['K15n160926', "JSJ[('hyperbolic', 't11128')]"],
 ['K15n164338', "JSJ[('hyperbolic', 's906')]"]]

In [20]:
NOT_HYP=[x[0] for x in NOT_HYP]
NOT_HYP

['K3a1',
 'K4a1',
 'K5a1',
 'K5a2',
 'K6a3',
 'K7a4',
 'K7a6',
 'K7a7',
 'K8n1',
 'K8n3',
 'K8a11',
 'K8a18',
 'K9n5',
 'K9a27',
 'K9a36',
 'K9a40',
 'K9a41',
 'K10n13',
 'K10n21',
 'K10n29',
 'K10a75',
 'K10a117',
 'K11n139',
 'K11n141',
 'K11a247',
 'K11a343',
 'K11a362',
 'K11a363',
 'K11a367',
 'K12n121',
 'K12n582',
 'K12n721',
 'K12a803',
 'K12a1166',
 'K12a1287',
 'K13n469',
 'K13a3143',
 'K13n3521',
 'K13n3523',
 'K13n3594',
 'K13n3596',
 'K13n3663',
 'K13a4573',
 'K13n4587',
 'K13n4639',
 'K13a4843',
 'K13a4856',
 'K13a4873',
 'K13a4878',
 'K14n3611',
 'K14a12741',
 'K14a17730',
 'K14n18212',
 'K14n19265',
 'K14a19429',
 'K14n21881',
 'K14n21882',
 'K14n22073',
 'K14n22180',
 'K14n22185',
 'K14n22589',
 'K14n24553',
 'K14n26039',
 'K15n19499',
 'K15n40211',
 'K15n41185',
 'K15n43522',
 'K15n48968',
 'K15n51748',
 'K15a54894',
 'K15n59184',
 'K15n72303',
 'K15a78880',
 'K15a84844',
 'K15a84969',
 'K15a85213',
 'K15a85234',
 'K15a85257',
 'K15a85263',
 'K15n112477',
 'K15n112479

In [32]:
start_time = time.time()
equal_surgeries=[] 

for [pol,x] in ALEX_GROUP:
    if x[0] not in NOT_HYP:
        K=snappy.Manifold(x[0]+str((0,1)))
        for i in range(len(x)-2):
            for y in nhyp16:
                if y[0]==x[i]:
                    break
            L=snappy.Link(change_notation(y[2])).exterior()
            L.dehn_fill((0,1))
            w=better_is_isometric_to(K,L,index=1000,try_hard=True)
            if w==False:
                print('The following pair has wrong homologies:',K,L)
            if w==True:
                print('We verified a pair:',K,L)
                equal_surgeries.append(x[0],x[i])



print('------------------------------------------')
print('Number of equal surgeries we could verify:',len(equal_surgeries))
print('Time taken: %s hours ' % ((time.time() - start_time)/3600))

------------------------------------------
Number of equal surgeries we could verify: 0
Time taken: 0.12449810114171769 hours 


So it seems that we do not get an example. We need to restrict the others.

Compute volumes to obstruct.

work with non hyperbolic examples and obstruct by hyperbolicity of others.

use regina for remaining ones.

Also does not need to consider fig 8 and trefoil.